In [1]:
%autosave 0

Autosave disabled


In [2]:

import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy

#Define Path of Files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# path for the 'Data' folder provided by DP
dir_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data'

#Functions

In [ ]:
# generate workflow for file storage /  instance creation

## Volume Matrix Manipulation and Poisson Instances Generation


In [59]:
# create dataframes for a region everyday

def generate_region_volume(region_name):
  # path of folder
  dir_region_instances = dir_loc + '/Instances/' + region_name
  dir_region_volume = dir_loc + '/Volumes/'
  # dir_region_districts = dir_region_instances + '/Districts'

  # load post_object-to-route_pos_id and post_point-to-post_object files 
  po_file = pd.read_csv(dir_region_instances + '/post_order_id_mapping.dat', sep='\t', names=('PostObjectId', 'RoutePosID'))
  pp_file = pd.read_csv(dir_region_instances + '/post_point_information.dat', sep='\t', names=('PostPointId', 'PostObjectId'))

  # pp_file adjustment by splitting list of post object ids
  pp_file['PostObjectId'] = pp_file['PostObjectId'].apply(literal_eval)
  pp_file = pp_file.explode('PostObjectId', ignore_index=True)

  # complete  post object list-district mapping
  region_district_df = generate_region_district(region_name = region_name)
  region_district_df.rename(columns = {'PostObjectID' : 'PostObjectId'}, inplace = True)

  # list of volume file paths
  day_names = []
  vol_path_list = []
  vol_day_map = {}
  day_map = {'mo' : 'Monday',
             'di' : 'Tuesday',
             'mi' : 'Wednesday',
             'do' : 'Thursday',
             'fr' : 'Friday',
             'sa' : 'Saturday'}

  for filename in os.listdir(dir_region_volume):
    vol_path_list.append(dir_region_volume + filename)
    day = filename[-6:-4]
    vol_day_map[filename] = day_map[day]

  # store dataframes of a region, map with post object id
  region_vol_day_dict = {}
  for vol_path in vol_path_list:
    vol_df = pd.read_csv(vol_path, sep = ';')
    vol_df.rename(columns = {'BRIEFE' : 'LETTERS',
                             'PAKETE' : 'PACKAGES',
                             'SONSTIGE' : 'OTHERS',
                             'ROUTEPOS_ID' : 'RoutePosID'},
                  inplace = True)
    
    # combining files to a complete table for a region
    vol_po_df = pd.merge(po_file, vol_df, on = 'RoutePosID', how = 'left')
    vol_po_df = pd.merge(pp_file, vol_po_df, on='PostObjectId', how='right')
    vol_po_df = pd.merge(region_district_df, vol_po_df, on='PostObjectId', how='right')

    # store dataframes in dict
    day_key = vol_day_map[vol_path[-18:]]
    region_vol_day_dict[day_key] = vol_po_df
    
  return region_vol_day_dict

In [47]:
def generate_region_district(region_name):
  district_path = dir_region_instances = dir_loc + '/Instances/' + region_name + '/Districts'
  region_districts_list = []

  for filename in os.listdir(district_path):
    file_var = re.sub('.dat', '', filename)
    file_district = pd.read_csv(district_path + '/' + filename, sep='\t', skiprows = [0,1],
                                names=('PostObjectID', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5')
                                # usecols = [0]
                                )
    file_district['district'] = file_var
    region_districts_list.append(file_district)

  region_districts_df = pd.concat(region_districts_list, ignore_index = True)
  region_districts_df.drop(['dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5'], axis = 1, inplace = True)
  
  return region_districts_df

In [29]:
# generate instances

def generate_instances(region_vol_day, scenario_type, scenario_method, scenario_number, growth_factor):
  # df = region_vol_day.copy() #.copy() used to avoid recopying on the original dataframe
  df = region_vol_day

  sce_letters = 'scenario_' + str(scenario_number) + '_letter'
  sce_packages = 'scenario_' + str(scenario_number) + '_package'
  sce_others = 'scenario_' + str(scenario_number) + '_others'

  df[sce_letters] = df['LETTERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate = growth_factor ))
  df[sce_packages] = df['PACKAGES'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate = growth_factor))
  df[sce_others] = df['OTHERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate = growth_factor))

  sce_all = 'scenario_' + str(scenario_number) + '_all'
  df[sce_all] = df[sce_letters] + df[sce_packages] + df[sce_others]

  return df

In [38]:
# # generate instances
# VERSION 2

# def generate_instances_v2(region_vol_day, scenario_method, growth_factor):
#   # df = region_vol_day.copy() #.copy() used to avoid recopying on the original dataframe
#   df = region_vol_day

#   scenario_type = random_poisson_instances_v2(method = scenario_method, rate = growth_factor) --> 7, 9, 10


#   df['sce_letters_low'], df['sce_letters_medium'], df['sce_letters_high'] = zip(*df['LETTERS'].map(scenario_type))


#   # sce_all = 'scenario_' + str(scenario_number) + '_all'
#   # df[sce_all] = df[sce_letters] + df[sce_packages] + df[sce_others]

#   return df

In [30]:
# generate real instances based on poisson

def random_poisson_instances(pos_delivery, method, rate):
  rng = np.random.default_rng()
  poisson_dist = rng.poisson(lam = pos_delivery * (1 + rate), size = 52)

  if method == 'random':
    return random.choice(poisson_dist)
  if method == 'mode':
    return scipy.stats.mode(poisson_dist, keepdims = True)[0][0]
  else:
    return 'only options : [random, mode]'

In [34]:
# # generate real instances based on poisson
# VERSION 2

# def random_poisson_instances_v2(pos_delivery, method, rate):
#   rng = np.random.default_rng()
#   poisson_dist_low = rng.poisson(lam = pos_delivery * (1 - rate), size = 52)
#   poisson_dist_med = rng.poisson(lam = pos_delivery, size = 52)
#   poisson_dist_high = rng.poisson(lam = pos_delivery * (1 + rate), size = 52)

#   if method == 'random':
#     return random.choice(poisson_dist_low), random.choice(poisson_dist_med), random.choice(poisson_dist_high)
#   if method == 'mode':
#     return scipy.stats.mode(poisson_dist, keepdims = True)[0][0]
#   else:
#     return 'only options : [random, mode]'

#Process Examples (Using Functions)

In [60]:
%%time
# ['Warmsen', 'Uerze', 'Hannover 92']

region_vol_day_dict = generate_region_volume(region_name = "Warmsen")

CPU times: user 12.9 s, sys: 1.16 s, total: 14 s
Wall time: 15.3 s


In [62]:
region_vol_day_dict['Monday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,0.45,0.293333,0.0
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,0.45,0.293333,0.0
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,0.45,0.293333,0.0
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.00,0.000000,0.0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.00,0.000000,0.0


In [65]:
%%time
#Creating the complete dictionary of volumes for a region
warmsen_instances = {}

#INPUT Rate of mail change 
rate = 0.5
rate_mapping = {'low' : -rate, 'medium' : 0, 'high' : rate}

# create instances for one region
for key in region_vol_day_dict.keys():
  # create instances per day
  for key_rate in rate_mapping:
    dummy = generate_instances(region_vol_day = region_vol_day_dict[key],
                               scenario_type = random_poisson_instances,
                               scenario_method = 'random',
                               scenario_number = key_rate,
                               growth_factor = rate_mapping[key_rate])
  warmsen_instances[key] = dummy

CPU times: user 14.7 s, sys: 338 ms, total: 15 s
Wall time: 15.2 s


In [67]:
warmsen_instances['Tuesday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS,scenario_low_letter,scenario_low_package,scenario_low_others,scenario_low_all,scenario_medium_letter,scenario_medium_package,scenario_medium_others,scenario_medium_all,scenario_high_letter,scenario_high_package,scenario_high_others,scenario_high_all
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,5.796667,0.69,0.273333,1,0,0,1,9,1,0,10,9,2,0,11
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,5.796667,0.69,0.273333,7,0,0,7,8,1,1,10,10,0,0,10
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,5.796667,0.69,0.273333,2,0,0,2,5,1,0,6,9,0,1,10
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.000000,0.00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.000000,0.00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
print(warmsen_instances['Tuesday']['scenario_low_all'].sum())
print(warmsen_instances['Tuesday']['scenario_medium_all'].sum())
print(warmsen_instances['Tuesday']['scenario_high_all'].sum())

4522
9014
13672


In [72]:
z = warmsen_instances['Tuesday']


print(z[z['district'] == '31606-14']['scenario_low_all'].sum())
print(z[z['district'] == '31606-14']['scenario_medium_all'].sum())
print(z[z['district'] == '31606-14']['scenario_high_all'].sum())

274
511
836


In [ ]:
region_name = 'Warmsen'

for key in warmsen_instances.keys():
  df_day = warmsen_instances[key]
  for district in df_day['district'].value_counts().index.tolist():
    print(district)
    df_day_district = df_day[df_day['district'] == district]

    district_path = dir_loc + '/Instances/' + region_name + '/Districts/' + district + '.dat'
    rows_needed = [1]
    file_district = pd.read_csv(district_path, sep='\t', skiprows = lambda x : x not in rows_needed,
                                names=('dum_0', 'start_point', 'end_point', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5', 'dum_6')
                                )
    start_point = file_district['start_point'][0]
    end_point = file_district['end_point'][0]

    points = [start_point, end_point]

    scenario_list = df_day_district.columns.tolist()[7:]
    for scenario in scenario_list:
      col_use = ['PostPointId']
      col_use.append(scenario)
      df_day_district_scenario = df_day_district[col_use]
      
      df_day_district_scenario_filtered = df_day_district_scenario[df_day_district_scenario[scenario] != 0]
      pp_id_day_district_scenario = df_day_district_scenario_filtered['PostPointId'].unique().tolist()
      
      for point in points:
        if point not in pp_id_day_district_scenario:
          pp_id_day_district_scenario.append(point)

      distance_path = dir_loc + '/Instances/' + region_name + '/distances'
      district_distance = pd.read_csv(distance_path + '/distances_' + district + '.dat',
                                  names=['pp_1', 'pp_2', 'dist']
                                  )
      district_distance_filtered = district_distance[(district_distance['pp_1'].isin(pp_id_day_district_scenario)) & (district_distance['pp_2'].isin(pp_id_day_district_scenario))]
      distance_matrix_df = district_distance_filtered.pivot(index = 'pp_1', columns = 'pp_2', values = 'dist')

      # distance matrix
      distance_matrix_array = distance_matrix_df.to_numpy()

      # node mapping
      map_val = list(range(0, len(distance_matrix_df)))
      nodes = distance_matrix_df.index.values.tolist()
      mapping = dict(zip(map_val, nodes))

      


31603-06
31600-01
31606-12
31600-02
31604-09
31603-07
31604-10
31600-03
31603-05
31603-08
31606-11
31606-14
31606-13
31600-04
31603-06
31600-01
31606-12
31600-02
31604-09
31603-07
31604-10
31600-03
31603-05
31603-08
31606-11
31606-14
31606-13
31600-04
31603-06
31600-01
